In [26]:
import pandas as pd
import numpy as np

In [27]:
with open('./data/CAN traffic (normal only).txt', 'r') as f:
  normal_data_list = [line.rstrip().split("\t") for line in f]

In [28]:
with open('./data/CAN traffic (attack included).txt', 'r') as f:
  abnormal_data_list = [line.rstrip().split("\t") for line in f]

In [29]:
normal_data_df = pd.DataFrame(normal_data_list, columns=['Timestamp', 'Arbitration ID', 'DLC', 'data', 'Class'])
abnormal_data_df = pd.DataFrame(abnormal_data_list, columns=['Timestamp', 'Arbitration ID', 'DLC', 'data', 'Class', 'SubClass'])

In [30]:
normal_data_df.head()

,Timestamp,Arbitration ID,DLC,data,Class
0,1597707827,260,8,06 25 05 30 FF CF 71 55,Normal
1,1597707827,329,8,4A C5 7E 8C 31 2D 01 10,Normal
2,1597707827,38D,8,00 00 49 00 90 7F FE 01,Normal
3,1597707827,420,8,50 1E 00 C8 FC 4F 6A 00,Normal
4,1597707827,421,8,FE 07 00 FF E3 7F 00 52,Normal


In [31]:
abnormal_data_df.head()

,Timestamp,Arbitration ID,DLC,data,Class,SubClass
0,1597759710,153,8,20 A1 10 FF 00 FF 50 1F,Normal,Normal
1,1597759710,220,8,13 24 7F 60 05 FF BF 10,Normal,Normal
2,1597759710,507,4,08 00 00 01,Normal,Normal
3,1597759710,356,8,00 00 00 80 16 00 00 00,Normal,Normal
4,1597759710,340,8,FC 03 00 E4 B7 21 FA 3C,Normal,Normal


**Using Hamming Distance**

In [32]:
normal_data_df['data'] = normal_data_df['data'].str.replace(' ','')
abnormal_data_df['data'] = abnormal_data_df['data'].str.replace(' ','')

In [33]:
normal_data_df.head()

,Timestamp,Arbitration ID,DLC,data,Class
0,1597707827,260,8,06250530FFCF7155,Normal
1,1597707827,329,8,4AC57E8C312D0110,Normal
2,1597707827,38D,8,00004900907FFE01,Normal
3,1597707827,420,8,501E00C8FC4F6A00,Normal
4,1597707827,421,8,FE0700FFE37F0052,Normal


**Normal Dataset 가공**

In [34]:
[normal_data_df['data'].loc[2][i:i+2] for i in range(0, len(normal_data_df['data'].loc[2]), 2)]

['00', '00', '49', '00', '90', '7F', 'FE', '01']

In [35]:
len(normal_data_df)

179346

In [36]:
normal_data_df['splited_data'] = 0

In [37]:
normal_data_df

,Timestamp,Arbitration ID,DLC,data,Class,splited_data
0,1597707827,260,8,06250530FFCF7155,Normal,0
1,1597707827,329,8,4AC57E8C312D0110,Normal,0
2,1597707827,38D,8,00004900907FFE01,Normal,0
3,1597707827,420,8,501E00C8FC4F6A00,Normal,0
4,1597707827,421,8,FE0700FFE37F0052,Normal,0
...,...,...,...,...,...,...
179341,1597707902,391,8,00000000000008EB,Normal,0
179342,1597707902,260,8,06391A30FFD1A163,Normal,0
179343,1597707902,421,8,FE0700FFE37F009E,Normal,0
179344,1597707902,130,8,948EF08100000BAC,Normal,0


In [38]:
len(normal_data_df['data'].loc[0])

16

In [39]:
def convert(x):
  if len(x) < 16:
    while len(x) < 16:
      x = '00' + x
    return x
  else:
    return x


In [40]:
normal_data_df['converted_data'] = normal_data_df['data'].apply(convert)

In [41]:
normal_data_df['converted_data']

0         06250530FFCF7155
1         4AC57E8C312D0110
2         00004900907FFE01
3         501E00C8FC4F6A00
4         FE0700FFE37F0052
                ...       
179341    00000000000008EB
179342    06391A30FFD1A163
179343    FE0700FFE37F009E
179344    948EF08100000BAC
179345    3080006F20000B35
Name: converted_data, Length: 179346, dtype: object

In [42]:
normal_data_df['converted_data'].loc[14]

'0000000000081CFA'

In [44]:
for j in range(len(normal_data_df)):
  normal_data_df['splited_data'].loc[j] = [normal_data_df['converted_data'].loc[j][i:i+2] for i in range(0, len(normal_data_df['converted_data'].loc[j]), 2)]

In [45]:
normal_data_df['splited_data'].loc[14]

['00', '00', '00', '00', '00', '08', '1C', 'FA']

In [46]:
from scipy.spatial.distance import hamming

In [47]:
x = normal_data_df['splited_data'].loc[0]
y = normal_data_df['splited_data'].loc[1]
hamming(x, y) * len(x)

8.0

In [48]:
normal_data_df['hamming_distance'] = 0

In [49]:
normal_data_df

,Timestamp,Arbitration ID,DLC,data,Class,splited_data,converted_data,hamming_distance
0,1597707827,260,8,06250530FFCF7155,Normal,"[06, 25, 05, 30, FF, CF, 71, 55]",06250530FFCF7155,0
1,1597707827,329,8,4AC57E8C312D0110,Normal,"[4A, C5, 7E, 8C, 31, 2D, 01, 10]",4AC57E8C312D0110,0
2,1597707827,38D,8,00004900907FFE01,Normal,"[00, 00, 49, 00, 90, 7F, FE, 01]",00004900907FFE01,0
3,1597707827,420,8,501E00C8FC4F6A00,Normal,"[50, 1E, 00, C8, FC, 4F, 6A, 00]",501E00C8FC4F6A00,0
4,1597707827,421,8,FE0700FFE37F0052,Normal,"[FE, 07, 00, FF, E3, 7F, 00, 52]",FE0700FFE37F0052,0
...,...,...,...,...,...,...,...,...
179341,1597707902,391,8,00000000000008EB,Normal,"[00, 00, 00, 00, 00, 00, 08, EB]",00000000000008EB,0
179342,1597707902,260,8,06391A30FFD1A163,Normal,"[06, 39, 1A, 30, FF, D1, A1, 63]",06391A30FFD1A163,0
179343,1597707902,421,8,FE0700FFE37F009E,Normal,"[FE, 07, 00, FF, E3, 7F, 00, 9E]",FE0700FFE37F009E,0
179344,1597707902,130,8,948EF08100000BAC,Normal,"[94, 8E, F0, 81, 00, 00, 0B, AC]",948EF08100000BAC,0


In [50]:
for i in range(len(normal_data_df) - 1):
  normal_data_df['hamming_distance'].loc[i] = hamming(normal_data_df['splited_data'].loc[i], normal_data_df['splited_data'].loc[i+1]) * len(normal_data_df['splited_data'].loc[i])

C:\Users\wlsrn\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [52]:
normal_data_df.head()

,Timestamp,Arbitration ID,DLC,data,Class,splited_data,converted_data,hamming_distance
0,1597707827,260,8,06250530FFCF7155,Normal,"[06, 25, 05, 30, FF, CF, 71, 55]",06250530FFCF7155,8.0
1,1597707827,329,8,4AC57E8C312D0110,Normal,"[4A, C5, 7E, 8C, 31, 2D, 01, 10]",4AC57E8C312D0110,8.0
2,1597707827,38D,8,00004900907FFE01,Normal,"[00, 00, 49, 00, 90, 7F, FE, 01]",00004900907FFE01,8.0
3,1597707827,420,8,501E00C8FC4F6A00,Normal,"[50, 1E, 00, C8, FC, 4F, 6A, 00]",501E00C8FC4F6A00,7.0
4,1597707827,421,8,FE0700FFE37F0052,Normal,"[FE, 07, 00, FF, E3, 7F, 00, 52]",FE0700FFE37F0052,7.0


**abnormal dataset 가공**

In [53]:
abnormal_data_df['splited_data'] = 0

In [54]:
abnormal_data_df['converted_data'] = abnormal_data_df['data'].apply(convert)

In [55]:
for j in range(len(abnormal_data_df)):
  abnormal_data_df['splited_data'].loc[j] = [abnormal_data_df['converted_data'].loc[j][i:i+2] for i in range(0, len(abnormal_data_df['converted_data'].loc[j]), 2)]

C:\Users\wlsrn\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [57]:
abnormal_data_df['splited_data']

0         [20, A1, 10, FF, 00, FF, 50, 1F]
1         [13, 24, 7F, 60, 05, FF, BF, 10]
2         [00, 00, 00, 00, 08, 00, 00, 01]
3         [00, 00, 00, 80, 16, 00, 00, 00]
4         [FC, 03, 00, E4, B7, 21, FA, 3C]
                        ...               
806385    [00, 3B, 28, 0B, 3B, 30, 00, 01]
806386    [00, 00, 00, 00, 05, 00, 00, 00]
806387    [00, 00, 00, 00, 01, 28, 0B, 42]
806388    [04, 7F, FF, FF, 00, 7B, 00, 26]
806389    [00, 00, 00, 00, 00, 00, 00, 00]
Name: splited_data, Length: 806390, dtype: object

In [58]:
abnormal_data_df['hamming_distance'] = 0

In [59]:
for i in range(len(abnormal_data_df) - 1):
  abnormal_data_df['hamming_distance'].loc[i] = hamming(abnormal_data_df['splited_data'].loc[i], abnormal_data_df['splited_data'].loc[i+1]) * len(abnormal_data_df['splited_data'].loc[i])

C:\Users\wlsrn\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [60]:
abnormal_data_df['hamming_distance']

0         7.0
1         8.0
2         3.0
3         7.0
4         8.0
         ... 
806385    6.0
806386    4.0
806387    8.0
806388    6.0
806389    0.0
Name: hamming_distance, Length: 806390, dtype: float64

In [91]:
def get_splited_arbid_seq_by_wnd(hamming_seq, window_size=3):
  splited = np.lib.stride_tricks.as_strided(hamming_seq, shape=(np.size(hamming_seq)-window_size+1, window_size), strides=(hamming_seq.strides[0],hamming_seq.strides[0]))
  return splited

In [92]:
train_ham_seq = normal_data_df['hamming_distance'].to_numpy()
test_ham_seq = abnormal_data_df['hamming_distance'].to_numpy()

In [93]:
print(len(train_ham_seq))
print(len(test_ham_seq))

179346
806390


In [94]:
tr_set = get_splited_arbid_seq_by_wnd(train_ham_seq)
te_set = get_splited_arbid_seq_by_wnd(test_ham_seq)

In [95]:
import hmmlearn.hmm as hmm

In [96]:
print(len(tr_set))
print(len(te_set))

179344
806388


In [97]:
tr_set_size = len(tr_set)
te_set_size = len(te_set)

In [1]:
h = hmm.GaussianHMM(2) # 모델 파라미터 설정, 모델링 객체 생성
print('--------------------')
h.fit(tr_set) # 학습진행
print(h.score(tr_set)) # log(foward prob)
print(h.score(te_set)/(te_set_size/tr_set_size))
print('--------------------')
print(h.startprob_)
print(h.transmat_)

NameError: name 'hmm' is not defined

In [99]:
h = hmm.GaussianHMM(3) # 모델 파라미터 설정, 모델링 객체 생성
print('--------------------')
h.fit(tr_set) # 학습진행
print(h.score(tr_set)) # log(foward prob)
print(h.score(te_set)/(te_set_size/tr_set_size))
print('--------------------')
print(h.startprob_)
print(h.transmat_)

--------------------
-826898.8899274276
-870018.7437751665
--------------------
[1.00000000e+00 5.55273230e-15 4.78199402e-29]
[[8.29994249e-01 4.62881512e-07 1.70005289e-01]
 [5.57901790e-01 3.77006758e-01 6.50914517e-02]
 [2.63255709e-08 5.00290597e-01 4.99709377e-01]]


In [72]:
h = hmm.GaussianHMM(4) # 모델 파라미터 설정, 모델링 객체 생성
print('--------------------')
h.fit(tr_set) # 학습진행
print(h.score(tr_set)) # log(foward prob)
print(h.score(te_set)/(te_set_size/tr_set_size))
print('--------------------')
print(h.startprob_)
print(h.transmat_)

--------------------
-1374846.1310721096
-1454209.9680619794
--------------------
[1.00000000e+00 3.28177901e-27 6.62318111e-15 5.65322935e-38]
[[8.22765704e-01 1.77233184e-01 1.11176760e-06 1.02686214e-10]
 [2.29333907e-06 4.55433407e-01 5.67724543e-10 5.44564299e-01]
 [4.29041859e-01 1.26268249e-01 4.44689875e-01 1.66387946e-08]
 [2.99172079e-11 1.12381937e-03 5.18401162e-01 4.80475019e-01]]


In [ ]:
V = 